In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing

raw_csv_data=np.loadtxt(r'C:\Users\admin\Desktop\Audiobooks_data.csv',delimiter=',')

unscaled_inputs_all=raw_csv_data[:,1:-1]
targets_all=raw_csv_data[:,-1]

In [2]:
num_one_inputs=int(np.sum(targets_all))
zero_targets_counter=0
indices_to_remove=[]

for i in range(targets_all.shape[0]):
    if targets_all[i]==0:
        zero_targets_counter+=1
        if zero_targets_counter>num_one_inputs:
            indices_to_remove.append(i)
            
unscaled_input_equal_priors=np.delete(unscaled_inputs_all,indices_to_remove,axis=0)
targets_equal_priors=np.delete(targets_all,indices_to_remove,axis=0)

In [3]:
scaled_inputs=preprocessing.scale(unscaled_input_equal_priors)

In [4]:
shuffled_indices=np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs=scaled_inputs[shuffled_indices]
shuffled_targets=targets_equal_priors[shuffled_indices]

In [5]:
samples_count=shuffled_inputs.shape[0]

train_samples_count=int(0.8*samples_count)
validation_samples_count=int(0.1*samples_count)
test_samples_count=samples_count-train_samples_count-validation_samples_count

train_inputs=shuffled_inputs[:train_samples_count]
train_targets=shuffled_targets[:train_samples_count]

validation_inputs=shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets=shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs=shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets=shuffled_targets[train_samples_count+validation_samples_count:]

In [6]:
np.savez('Audiobooks_data_train',inputs=train_inputs,targets=train_targets)
np.savez('Audiobooks_data_validation',inputs=validation_inputs,targets=validation_targets)
np.savez('Audiobooks_data_test',inputs=test_inputs,targets=test_targets)

In [7]:
npz=np.load('Audiobooks_data_train.npz')

train_inputs=npz['inputs'].astype(float)
train_targets=npz['targets'].astype(int)

npz1=np.load('Audiobooks_data_validation.npz')

validation_inputs=npz1['inputs'].astype(float)
validation_targets=npz1['targets'].astype(int)

npz2=np.load('Audiobooks_data_test.npz')

test_inputs=npz2['inputs'].astype(float)
test_targets=npz2['targets'].astype(int)


In [12]:
input_size=10
output_size=2
hidden_layer_size=50

model=tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
    tf.keras.layers.Dense(output_size,activation='softmax'),
])

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

max_epochs=100

early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
         train_targets,
         batch_size=100,
         epochs=max_epochs,
         callbacks=[early_stopping],
         validation_data=(validation_inputs,validation_targets),
         verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 0.5129 - accuracy: 0.8220 - val_loss: 0.4068 - val_accuracy: 0.8770
Epoch 2/100
3579/3579 - 0s - loss: 0.3588 - accuracy: 0.8729 - val_loss: 0.3197 - val_accuracy: 0.8837
Epoch 3/100
3579/3579 - 0s - loss: 0.3114 - accuracy: 0.8840 - val_loss: 0.3036 - val_accuracy: 0.8881
Epoch 4/100
3579/3579 - 0s - loss: 0.2931 - accuracy: 0.8899 - val_loss: 0.2913 - val_accuracy: 0.8926
Epoch 5/100
3579/3579 - 0s - loss: 0.2782 - accuracy: 0.8933 - val_loss: 0.2833 - val_accuracy: 0.8971
Epoch 6/100
3579/3579 - 0s - loss: 0.2688 - accuracy: 0.8980 - val_loss: 0.2790 - val_accuracy: 0.8949
Epoch 7/100
3579/3579 - 0s - loss: 0.2615 - accuracy: 0.9016 - val_loss: 0.2738 - val_accuracy: 0.8971
Epoch 8/100
3579/3579 - 0s - loss: 0.2564 - accuracy: 0.9039 - val_loss: 0.2693 - val_accuracy: 0.9060
Epoch 9/100
3579/3579 - 0s - loss: 0.2510 - accuracy: 0.9047 - val_loss: 0.2698 - val_accuracy: 0.8993
Epoch 10/100
3579/3579 - 0

In [13]:
test_loss,test_accuracy=model.evaluate(test_inputs,test_targets)

448/448 [==============================] - 0s 350us/sample - loss: 0.2579 - accuracy: 0.9129
